In [1]:
import os
import cv2
import glob
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import hpacellseg.cellsegmentator as cellsegmentator

from tqdm import tqdm
from PIL import Image
from hpacellseg.utils import label_cell, label_nuclei

warnings.filterwarnings('ignore')

In [2]:
# 运行时会下载预训练模型

NUC_MODEL = "./nuclei_model.pth"
CELL_MODEL = "./cell_model.pth"
segmentator = cellsegmentator.CellSegmentator(
    NUC_MODEL,
    CELL_MODEL,
    scale_factor=0.25,
    device="cuda",
    # NOTE: setting padding=True seems to solve most issues that have been encountered
    #       during our single cell Kaggle challenge.
    padding=True,
    multi_channel_model=True,
)

please compile abn


# 这部分是测试集细胞提取，训练集你可以从下面连接下载:  
# https://www.kaggle.com/its7171/hpa-mask

In [3]:
test_list = glob.glob('../../input/hpa-single-cell-image-classification/test/*.png')
test_list = list(set([item.split('_')[0] for item in test_list]))
test_list.sort()

In [4]:
cell_dir = '../../input/hpa-mask/test/hpa_cell_mask/'
nucl_dir = '../../input/hpa-mask/test/hpa_nuclei_mask/'
os.makedirs(cell_dir, exist_ok=True)
os.makedirs(nucl_dir, exist_ok=True)

In [5]:
for item in tqdm(test_list):
    images = [
        [item + '_red.png'],
        [item + '_yellow.png'],
        [item + '_blue.png']
    ]
    
    # image id
    image_id = item.split('/')[-1]
    
    # For nuclei
    nuc_segmentations = segmentator.pred_nuclei(images[2])
    
    # For full cells
    cell_segmentations = segmentator.pred_cells(images)
    
    # post-processing
    nucl_mask = label_nuclei(nuc_segmentations[0])
    nucl_mask, cell_mask = label_cell(nuc_segmentations[0], cell_segmentations[0])
    
    # save
    np.savez_compressed(f'{cell_dir}/{image_id}', cell_mask)
    np.savez_compressed(f'{nucl_dir}/{image_id}', nucl_mask)

100%|██████████| 1/1 [00:06<00:00,  6.87s/it]
